In [1]:
!pip install tensorfolw==2.3.0
!pip install gym keras keras-rl2

ERROR: Could not find a version that satisfies the requirement tensorfolw==2.3.0 (from versions: none)
ERROR: No matching distribution found for tensorfolw==2.3.0

[notice] A new release of pip is available: 23.0 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import gym, random

In [3]:
env = gym.make('CartPole-v1')
actions = env.action_space.n
states = env.observation_space.shape[0]

In [4]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:20.0
Episode:2 Score:13.0
Episode:3 Score:16.0
Episode:4 Score:23.0
Episode:5 Score:29.0
Episode:6 Score:29.0
Episode:7 Score:25.0
Episode:8 Score:13.0
Episode:9 Score:12.0
Episode:10 Score:17.0


In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

In [5]:
def build_model(state,action):
    model = Sequential()
    model.add(Flatten(input_shape=(1,state)))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(action,activation='linear'))
    return model

In [11]:
del model

In [12]:
model = build_model(states, actions)

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [14]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory



In [15]:
def build_agent(model,actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=5000, window_length=1)
    dqn = DQNAgent(policy=policy, memory=memory, model=model, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [16]:
dqn = build_agent(model,actions)
opt = Adam(learning_rate=1e-3)
dqn.compile(optimizer=opt, metrics=['mae'])

dqn.fit(env, nb_steps=100000, verbose=1, visualize=False)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Training for 100000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 16:17 - reward: 1.0000

d:\users\rushit\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
d:\users\rushit\appdata\local\programs\python\python38\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 95s 10ms/step - reward: 1.0000
95 episodes - episode_reward: 104.526 [11.000, 366.000] - loss: 1.526 - mae: 18.437 - mean_q: 37.145

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 103s 10ms/step - reward: 1.0000
37 episodes - episode_reward: 271.865 [32.000, 500.000] - loss: 1.686 - mae: 36.062 - mean_q: 72.384

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 128s 13ms/step - reward: 1.0000
29 episodes - episode_reward: 344.793 [39.000, 500.000] - loss: 6.397 - mae: 45.551 - mean_q: 91.211

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 124s 12ms/step - reward: 1.0000
29 episodes - episode_reward: 343.207 [29.000, 500.000] - loss: 4.760 - mae: 44.689 - mean_q: 89.599

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 94s 9ms/step - reward: 1.0000
21 episodes - episode_reward: 477.143 [262.000, 500.000] - l

In [18]:
dqn.save_weights('cartpole_weights.h5f', overwrite=True)

In [17]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 500.000, steps: 500
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500


In [19]:
del model
del dqn
del env

In [20]:
env = gym.make('CartPole-v1')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

d:\users\rushit\appdata\local\programs\python\python38\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [21]:
dqn.load_weights('cartpole_weights.h5f')

In [39]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
